<a href="https://colab.research.google.com/github/Tusharprabhu/HardwareTrojanDetection/blob/main/verilogtrojan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt


In [ ]:
def load_model():
    print("Loading tokenizer and model...")
    tokenizer = AutoTokenizer.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog")
    model = AutoModelForCausalLM.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog")
    return tokenizer, model

In [ ]:
def calculate_perplexity(text, tokenizer, model):
    # Truncate text if it's too long for the model
    max_length = 512  # Set a reasonable value
    input_ids = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).input_ids
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [ ]:

def analyze_alu_files(clean_file_path, suspicious_file_path):
    tokenizer, model = load_model()


    print(f"Analyzing clean ALU file: {clean_file_path}")
    with open(clean_file_path, 'r', encoding='utf-8') as f:
        clean_code = f.read()

    clean_perplexity = calculate_perplexity(clean_code, tokenizer, model)
    print(f"Clean ALU perplexity: {clean_perplexity:.4f}")

    print(f"\nAnalyzing suspicious ALU file: {suspicious_file_path}")
    with open(suspicious_file_path, 'r', encoding='utf-8') as f:
        suspicious_code = f.read()

    suspicious_perplexity = calculate_perplexity(suspicious_code, tokenizer, model)
    print(f"Suspicious ALU perplexity: {suspicious_perplexity:.4f}")


    difference = suspicious_perplexity - clean_perplexity
    percent_increase = (difference / clean_perplexity) * 100

    print(f"\nPerplexity difference: {difference:.4f}")
    print(f"Percent increase: {percent_increase:.2f}%")

    threshold_percent = 20
    if percent_increase > threshold_percent:
        print(f"Potential hardware trojan detected!")
        print(f"Perplexity increase exceeds {threshold_percent}%")
    else:
        print(f"\n✓ No significant anomalies detected (threshold: {threshold_percent}%)")


    plot_comparison(clean_perplexity, suspicious_perplexity)

    return clean_perplexity, suspicious_perplexity

In [ ]:
def plot_comparison(clean_perplexity, suspicious_perplexity):
    plt.figure(figsize=(8, 5))

    bars = plt.bar(['Clean ALU', 'Suspicious ALU'],
                   [clean_perplexity, suspicious_perplexity],
                   color=['green', 'red'], alpha=0.7)

    plt.ylabel('Perplexity (lower is more expected)')
    plt.title('Perplexity Comparison: Clean vs. Suspicious ALU Code')
    plt.grid(axis='y', alpha=0.3)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.4f}', ha='center', va='bottom')

    plt.savefig('alu_perplexity_comparison.png')
    plt.close()
    print("Saved perplexity comparison plot to 'alu_perplexity_comparison.png'")


In [ ]:
if __name__ == "__main__":
    clean_alu_path = "alu_clean.v"
    suspicious_alu_path = "alu_suspicious.v"

    analyze_alu_files(clean_alu_path, suspicious_alu_path)

Loading tokenizer and model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at shailja/fine-tuned-codegen-2B-Verilog were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.20.attn.causal_mask', 'transformer.h.21.attn.causal_mask', 'transformer.h.22.attn.causal_mask', 'transformer.h.23.attn.causal_mask', 'transformer.h.24.attn.causal_mask', 'transformer.h.25.attn.causal_mask', 'transformer.h.26.attn.causal_mask', 'transformer.h.27.attn.causal_mask', 'transformer.h.28.attn.causal_mask', 'transformer.h.29.attn.causal_mask', 'transformer.h.3.attn

Analyzing clean ALU file: alu_clean.v
Clean ALU perplexity: 2.0105

Analyzing suspicious ALU file: alu_suspicious.v
Suspicious ALU perplexity: 3.2570

Perplexity difference: 1.2465
Percent increase: 62.00%
Potential hardware trojan detected!
Perplexity increase exceeds 20%
Saved perplexity comparison plot to 'alu_perplexity_comparison.png'


# Hardware Trojan Detection using Language Models

This project demonstrates how to use a pre-trained language model to detect potential hardware trojans in Verilog code. The core idea is that a hardware trojan, being an unexpected and malicious modification, will likely have a higher perplexity score when analyzed by a language model trained on legitimate hardware designs.

## How it Works

1. **Language Model:** We use the `shailja/fine-tuned-codegen-2B-Verilog` model, which is a CodeGen model fine-tuned on Verilog code. This model has learned the statistical patterns of legitimate Verilog code.

2. **Perplexity:** Perplexity is a measure of how well a probability model predicts a sample. In our case, it measures how "surprising" a piece of Verilog code is to the language model. A lower perplexity score indicates that the code is more expected and "natural" according to the model's learned patterns.

3. **Trojan Detection:** We calculate the perplexity of a "clean" (known good) Verilog file and a "suspicious" Verilog file. A significantly higher perplexity score for the suspicious file suggests that it contains code that deviates from the norm, which could indicate the presence of a hardware trojan.

## How to Use

1. **Install dependencies:**

In [ ]:
   if __name__ == "__main__":
       clean_alu_path = "alu_clean.v"
       suspicious_alu_path = "alu_suspicious.v"

       analyze_alu_files(clean_alu_path, suspicious_alu_path)